In [8]:
import numpy as np
import cPickle
import nltk
from nltk.corpus import stopwords
from collections import Counter
from sklearn.metrics.pairwise import pairwise_distances
import itertools
import string
from nltk.tokenize import sent_tokenize
from nltk.stem.porter import *
import re
#import pyemd
#loading word embeddings
#wv = KeyedVectors.load_word2vec_format("../w2v/w2vemb.bin", binary=True)

# Visualize functions

In [9]:
def write_file(n_clusters_, labels, corpus, mapping_dict, test_corpus):
    for indice_cluster in range(n_clusters_):
        # print indice_cluster
        idx_list = np.where(labels == indice_cluster)[0]
        f1 = open('cluster_%s/%dth_cluster.txt' % (question_class, indice_cluster), 'w+')
        for idx in idx_list:
            f1.write('%s %s\n' % (corpus[mapping_dict[idx] - 1], test_corpus[idx]))
    f1.close()


def write_ne(nes, question_class, mapping_dict):
    f1 = open('ne_%s.txt' % question_class, 'w+')
    for idx, doc in enumerate(nes):
        f1.write('%s %s\n' % (mapping_dict[idx], ' '.join(doc)))
    f1.close()    

def wmd_compute(x1, x2):
    return wv.wmdistance(x1, x2)

def analyze_cluster(n_clusters_, labels, corpus, mapping_dict, test_corpus, filename):
    num = 0
    other_content = []
    for indice_cluster in range(n_clusters_):
        idx_list = np.where(labels == indice_cluster)[0]
        if len(idx_list) > 4:
            f1 = open('%s_%s/%dth_cluster.txt' % (filename, question_class, indice_cluster), 'w+')
            for idx in idx_list:
                f1.write('%s %s\n' % (corpus[mapping_dict[idx] - 1], test_corpus[idx]))
            f1.close()
        if len(idx_list) < 4:
            num = num + len(idx_list)
            for idx in idx_list:
                other_content.append(corpus[mapping_dict[idx] - 1])
    f1 = open('%s_%s/other_cluster.txt' % (filename, question_class), 'w+')
    for doc in other_content:
        f1.write('%s\n' % doc)
    f1.close()
    print num

# Reading Raw text

In [10]:
question_class = 'q10'
fname = '..//data//%s.txt' % question_class
with open(fname) as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content]
print 'length of comments', len(content)
f.close()

length of comments 445


In [11]:
def clean_corpus(x1):
    x_new = []
    for x in x1:
        if 'no improvement' in x:
            continue
        else:
            x_new.append(x)
    return x_new

def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Every dataset is lower cased except for TREC
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"\'s", " ", string) 
    string = re.sub(r"\'ve", " ", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " ", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " ", string) 
    string = re.sub(r"\)", " ", string) 
    string = re.sub(r"\?", " ", string) 
    string = re.sub(r"\s{2,}", " ", string)    
    return string.strip().lower()


def process_corpus(x1,stopwords):
    # split three categories: 1 no improvemnt 2 with noun 3 others
    doc_noimprove = []
    clean_sen = []
    for x in x1:
        if 'no improvement' in x or 'respondent' in x:
            doc_noimprove.append(x)
        else:
            sents = sent_tokenize(x)
            sen_list = []
            for sent in sents:
                word_list = nltk.word_tokenize(sent)
                word_list = [clean_str(word) for word in word_list if word not in stop_words]
                sen_list = sen_list + word_list
            sen_list = ' '.join(sen_list)
            
            clean_sen.append(sen_list.strip())
    return clean_sen

def should_nn_v(x1, pos_nn, pos_vv):
    pos_new = nltk.pos_tag(nltk.word_tokenize(x1))
    nn_list = []
    vv_list = []
    for token in pos_new:
        if token[1] in pos_nn: #and token[0] in wv.vocab:
            nn_list.append(token[0])
        if token[1] in pos_vv:
            vv_list.append(token[0])
    print nn_list
    print vv_list
    print x1
    
nn_corpus = []

stop_words = stopwords.words('english')
rule_10_words = ['follow', 'up','call', 'calls', 'give']
stop_words = stop_words + rule_10_words
new_content = process_corpus(content, stop_words)

#pos_nn = ['NN', 'NNS']
#pos_vv = ['VB']
#print 'no comments', len(doc1)
#print 'single sentence without nn', len(doc3)
#print 'multi-sentence', len(doc4)
#for doc_sing in doc2:
    #should_nn_v(doc_sing, pos_nn, pos_vv)
    

# Representation BOW

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(min_df=2, stop_words='english')

tfidf = tfidf_vectorizer.fit_transform(new_content)



# Representation NMF

In [18]:
from sklearn.decomposition import NMF
n_topics = 50
nmf_data = NMF(n_components=n_topics, random_state=1, alpha=.1, l1_ratio=.5).fit_transform(tfidf)
print nmf_data.shape

(426, 50)


# Representation Topic Model

In [21]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

from sklearn.decomposition import LatentDirichletAllocation
n_topics = 50
tm_model = LatentDirichletAllocation(n_topics, max_iter=100,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0).fit(tfidf)
print("\nTopics in LDA model:")
n_top_words=10
tf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(tm_model, tf_feature_names, n_top_words)
tm_data = tm_model.transform(tfidf)


Topics in LDA model:
Topic #0:
issue mentioned resolve issues customer technician mention situation arrange immediate
Topic #1:
customer shinrai car days feedback proper money toyota home asked
Topic #2:
service regarding dealership phone receive till conditions inform toyota things
Topic #3:
questions feedback know sent asking come car service improvement number
Topic #4:
parts proper irritating concentrate fallow number contact registered delivery actually
Topic #5:
clear information dealership good car service customer servicing week feedback
Topic #6:
job check details needs receive received update times implement feedback
Topic #7:
relationship car road understand visit came correctly number tell happened
Topic #8:
problem days vehicle day servicing car solve working delivery condition
Topic #9:
mail used process dealership available suppose improve tell send getting
Topic #10:
servicing feedback car customer service time problem ask dealership customers
Topic #11:
know satisfied

# Clustering

In [23]:
from sklearn.cluster import KMeans

def print_cluster_docs(model, content):
    for topic_idx in enumerate(model.n_clusters):
        print("Topic #%d:" % topic_idx)
        print '\n'
        for idx, label in enumerate(model_labels_):
            if label == topic_idx:
                print content[idx]
                print '\n'
true_k = 10
km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
km.fit(tfidf)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=10, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [25]:
print km.labels_

[0 5 1 0 0 6 3 9 6 0 8 8 4 8 0 6 9 3 2 8 0 3 5 4 0 1 3 9 1 6 5 4 6 2 0 1 0
 8 0 3 1 3 2 7 3 3 3 8 0 0 9 4 2 2 8 3 4 0 3 0 8 1 7 1 0 2 3 5 3 7 1 0 9 5
 0 9 0 3 6 5 4 7 3 8 0 6 3 0 3 4 4 5 3 5 6 8 0 1 0 5 1 1 2 1 1 8 0 1 6 1 3
 3 2 3 0 0 3 8 3 2 5 8 2 1 5 6 0 8 6 4 4 0 3 0 8 3 2 0 7 0 8 8 9 0 0 0 0 3
 6 6 5 0 8 5 7 1 1 5 0 3 8 0 1 6 8 2 6 3 0 3 5 0 8 2 0 3 0 5 8 0 9 3 0 0 9
 3 4 1 6 3 0 1 9 1 1 3 0 1 4 3 0 0 9 3 0 3 3 3 1 4 8 4 3 0 3 3 8 4 3 3 5 0
 9 1 1 0 9 9 4 3 0 8 1 2 1 5 2 0 1 8 1 7 0 0 3 2 2 3 8 8 9 3 0 3 0 6 0 0 4
 0 1 0 1 3 3 3 9 8 3 4 5 0 0 0 1 0 1 0 3 8 1 1 1 9 2 9 4 7 3 1 3 0 1 3 9 3
 3 0 1 1 1 9 0 8 6 2 3 0 6 1 3 3 3 1 1 3 3 8 3 4 5 3 9 8 6 8 0 9 3 1 3 5 4
 3 4 0 4 0 5 0 4 0 6 0 0 2 0 8 5 0 0 3 4 6 3 0 2 5 1 9 8 7 0 8 3 4 8 0 6 9
 3 6 0 9 4 2 3 2 3 1 8 2 0 3 3 3 1 3 8 4 0 3 4 1 0 1 3 1 4 8 3 2 0 3 2 0 3
 8 0 1 8 8 3 8 1 8 1 1 3 3 2 1 7 2 3 3]
